In [15]:
#install the necessary packages 
import numpy as np
import modern_robotics as mr 

In [23]:
def FeedbackControl(currentactEffector, currentrefEffector, nextrefEffector, pGain, iGain, dt):
    '''
    Parameters
    ----------
    currentactEffector: np.array
        An (4,4) shaped np.array representing the current actual end effector configuration
    
    currentrefEffector: np.array
        An (4,4) shaped np.array representing the current end effector reference configuration
    
    nextrefEffector: np.array
        The end-effector reference configuration at the next timestep in the reference trajectory
    
    pGain: np.array
        A (6,6) shaped array representing the proportional gain matrix.
    
    iGain: np.array
        A (6,6) shaped array representing the integral gain matrix.
    
    dt: float
        A number that represents the timestep between the reference trajectory configurations. 

    Output
    -------
    v: np.array
        The commanded end effector twist V expressed in the end effector frame. 
    ''' 
    #SOLVE FOR THE FEEDFORWARD CONTROL LAW
    #Solve for [AdX^-1X_d]
    invX = mr.TransInv(currentactEffector)
    adj = mr.Adjoint(invX @ currentrefEffector)

    #Solve for V_d 
    invX_d = mr.TransInv(currentrefEffector)
    vdse3 = 1/dt * mr.MatrixLog6(invX_d @ nextrefEffector)
    v_d = mr.se3ToVec(vdse3)

    #Solve for X_err
    xerr_se3 = mr.MatrixLog6(invX @ currentrefEffector)
    x_err = mr.se3ToVec(xerr_se3)

    #SOLVE FOR THE COMMANDED VECTOR
    v = adj @ v_d + pGain @ x_err + iGain @ (x_err * dt)
    
    return v 


In [30]:
x_d = np.array([[0, 0, 1, 0.5],
                [0, 1, 0, 0],
                [-1, 0, 0, 0.5],
                [0, 0, 0, 1]])

x_dnext = np.array([[0, 0, 1, 0.6],
                    [0, 1, 0, 0],
                    [-1, 0, 0, 0.3],
                    [0, 0, 0, 1]])

x = np.array([[0.170, 0, 0.985, 0.387],
              [0, 1, 0, 0],
              [-0.985, 0, 0.170, 0.570],
              [0, 0, 0, 1]])

Kp = np.identity((6))
Ki = np.zeros((6))
dt = 0.01

v = FeedbackControl(x, x_d, x_dnext, Kp, Ki, dt)

In [1]:
#SOLVE FOR THE COMMANDED WHEEL AND ARM TORQUES
#write the Body Jacobian 
blist = np.array([[    0,       0,       0,       0, 0],
                  [    0,      -1,      -1,      -1, 0],
                  [    1,       0,       0,       0, 1],
                  [    0, -0.5076, -0.3526, -0.2176, 0],
                  [0.033,       0,       0,       0, 0],
                  [    0,       0,       0,       0, 0]])

M_0e = np.array([[1, 0, 0, 0.033],
                 [0, 1, 0, 0],
                 [0, 0, 1, 0.6546],
                 [0, 0, 0, 1]])

thetalist = np.array([0,0,0.2,-1.6,0])

#will change based on the current angle configuration that its at
Jb = mr.JacobianBody(blist, thetalist)

#find the Base Jacobian
#hardcoded kinematics
r = 0.0475
l = 0.235
w = 0.15
F = (r/4) * np.array([[-1/(l+w), 1/(l+w), 1/(l+w), -1/(l+w)],
                      [       1,       1,       1,        1],
                      [      -1,       1,      -1,        1]])

#create the F6 vector
F6 = np.vstack((np.zeros((2,4)), F, np.zeros((1,4))))

#create the transformation we need to find the 
T_b0 = np.array([[1, 0, 0, 0.1662],
                 [0, 1, 0, 0],
                 [0, 0, 1, 0.0026],
                 [0, 0, 0, 1]])

invT_b0 = mr.TransInv(T_b0)
T_0e = mr.FKinBody(M_0e, blist, thetalist )
invT_0e = mr.TransInv(T_0e)

J_base = mr.Adjoint(invT_0e @ invT_b0) @ F6
J_e = np.hstack((J_base, Jb))

command = np.linalg.pinv(J_e) @ v 

NameError: name 'np' is not defined